In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from models.lstm_cnn_attention import LSTMCNNAttention

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [2]:
# Sanity check: does the model run?

model = LSTMCNNAttention()

x = torch.randn(8, 75, 3)  # batch of 8 samples
y = model(x)

print("Output shape:", y.shape)

Output shape: torch.Size([8, 3])


In [3]:
# Load pre-generated dataset
X_train = np.load("../data/X_train.npy")
y_train = np.load("../data/y_train.npy")

X_val = np.load("../data/X_val.npy")
y_val = np.load("../data/y_val.npy")

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)

Train shape: (10500, 75, 3) (10500,)
Val shape: (2250, 75, 3) (2250,)


In [4]:
# Convert to PyTorch tensors
'''
    Why?
    PyTorch models only work with tensors
    Labels must be long for classification
'''

X_train_t = torch.tensor(X_train, dtype = torch.float32)
y_train_t = torch.tensor(y_train, dtype = torch.long)

X_val_t = torch.tensor(X_val, dtype = torch.float32)
y_val_t = torch.tensor(y_val, dtype = torch.long)

In [5]:
# Model
model = LSTMCNNAttention()

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

# Training params
EPOCHS = 15
BATCH_SIZE = 64

''' Simple. No tuning yet '''

' Simple. No tuning yet '

In [9]:
def train_one_epoch(model, X, y, optimizer, criterion, batch_size):

    model.train()
    total_loss = 0
    correct = 0
    num_batches = (len(X) + batch_size - 1) // batch_size

    for i in range(0, len(X), batch_size):
        xb = X[i:i + batch_size]
        yb = y[i:i + batch_size]

        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim = 1)
        correct += (preds == yb).sum().item()

        # Print progress
        print(f"Batch {i//batch_size + 1}/{num_batches} - Loss: {loss.item():.4f}")

    acc = correct / len(X)
    return total_loss, acc

In [10]:
# Validation Loop

def evaluate(model, X, y, criterion):
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        outputs = model(X)
        loss = criterion(outputs, y)

        total_loss = loss.item()
        preds = outputs.argmax(dim = 1)
        correct = (preds == y).sum().item()

    acc = correct / len(X)
    return total_loss, acc

In [11]:
# Train Model

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(
        model, X_train_t, y_train_t, optimizer, criterion, BATCH_SIZE
    )

    val_loss, val_acc = evaluate(
        model, X_val_t, y_val_t, criterion
    )

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Acc: {train_acc:.3f} | "
        f"Val Acc: {val_acc:.3f}"
    )

Batch 1/165 - Loss: 1.0947
Batch 2/165 - Loss: 1.0827
Batch 3/165 - Loss: 1.0729
Batch 4/165 - Loss: 1.0632
Batch 5/165 - Loss: 1.0551
Batch 6/165 - Loss: 1.0462
Batch 7/165 - Loss: 1.0574
Batch 8/165 - Loss: 1.0200
Batch 9/165 - Loss: 1.0221
Batch 10/165 - Loss: 1.0223
Batch 11/165 - Loss: 1.0074
Batch 12/165 - Loss: 0.9991
Batch 13/165 - Loss: 0.9793
Batch 14/165 - Loss: 0.9649
Batch 15/165 - Loss: 0.9360
Batch 16/165 - Loss: 0.9489
Batch 17/165 - Loss: 0.8869
Batch 18/165 - Loss: 0.8412
Batch 19/165 - Loss: 0.8391
Batch 20/165 - Loss: 0.8198
Batch 21/165 - Loss: 0.7883
Batch 22/165 - Loss: 0.7652
Batch 23/165 - Loss: 0.7097
Batch 24/165 - Loss: 0.6749
Batch 25/165 - Loss: 0.6782
Batch 26/165 - Loss: 0.6861
Batch 27/165 - Loss: 0.6827
Batch 28/165 - Loss: 0.6918
Batch 29/165 - Loss: 0.5356
Batch 30/165 - Loss: 0.4843
Batch 31/165 - Loss: 0.5146
Batch 32/165 - Loss: 0.5993
Batch 33/165 - Loss: 0.4999
Batch 34/165 - Loss: 0.4695
Batch 35/165 - Loss: 0.4626
Batch 36/165 - Loss: 0.4756
B

On synthetic data, you should see:

- accuracy quickly rise above 90%
- validation track training closely

That confirms:
- dataset is usable
- model is learning
- pipeline is correct

In [12]:
# Load test data
X_test = np.load("../data/X_test.npy")
y_test = np.load("../data/y_test.npy")

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

print("Test shape:", X_test_t.shape, y_test_t.shape)

Test shape: torch.Size([2250, 75, 3]) torch.Size([2250])


In [ ]:
# Evaluate on test set
test_loss, test_acc = evaluate(
    model, X_test_t, y_test_t, criterion
)

print(f"Test Accuracy: {test_acc:.3f}")

Test Accuracy: 0.999


You should expect:
- Test accuracy ≈ validation accuracy
- Slight drop is okay

In [15]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report

model.eval()
with torch.no_grad():
    outputs = model(X_test_t)
    preds = outputs.argmax(dim=1).cpu().numpy()

cm = confusion_matrix(y_test, preds)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:")
print(classification_report(y_test, preds, target_names=[
    "Light Braking", "Normal Braking", "Emergency Braking"
]))

Confusion Matrix:
 [[776   3   0]
 [  0 711   0]
 [  0   0 760]]

Classification Report:
                   precision    recall  f1-score   support

    Light Braking       1.00      1.00      1.00       779
   Normal Braking       1.00      1.00      1.00       711
Emergency Braking       1.00      1.00      1.00       760

         accuracy                           1.00      2250
        macro avg       1.00      1.00      1.00      2250
     weighted avg       1.00      1.00      1.00      2250



In [16]:
# Save trained model
torch.save(model.state_dict(), "../models/lstm_cnn_attention_baseline.pth")
print("Model saved successfully.")

Model saved successfully.


Files after saving this model 

models/
- lstm_cnn_attention.py
- lstm_cnn_attention_baseline.pth
